In [1]:
import requests
import json
import pandas as pd
#helper function to get responses of the api and store each layer in a dictionary
def get_responses(*args):
    list_of_responses={}
    for parm in args:
        url = f"https://emotional.byteroad.net/collections/{parm}/items?f=json&lang=en-US&limit=10000&skipGeometry=false&offset=0"
        response = requests.get(url)
        
        if response.status_code == 200:
            data_dict = json.loads(response.content)
            list_of_responses[parm]=data_dict
        else:
            print("Error: API request unsuccessful.")
    return list_of_responses

#maps of physical environment 

response = get_responses('hex350_grid_ndvi2022')['hex350_grid_ndvi2022']['features']
response2 = get_responses('hex350_grid_pm25_2019')['hex350_grid_pm25_2019']['features']
response3 = get_responses('hex350_grid_pm10_2019')['hex350_grid_pm10_2019']['features']
response4 = get_responses('hex350_grid_no2_2019')['hex350_grid_no2_2019']['features']
response5 = get_responses('hex350_grid_laeq16h')['hex350_grid_laeq16h']['features']
response6 = get_responses('hex350_grid_flood_4band')['hex350_grid_flood_4band']['features']
response7 = get_responses('hex350_grid_avglst')['hex350_grid_avglst']['features']
response8 = get_responses('hex350_grid_cycleroutes2021')['hex350_grid_cycleroutes2021']['features']
response9 = get_responses('hex350_grid_hindex2020')['hex350_grid_hindex2020']['features']
response10 = get_responses('hex350_grid_access2openspace')['hex350_grid_access2openspace']['features']

#maps of physical health

response11 = get_responses('hex350_grid_cardio_1920')['hex350_grid_cardio_1920']['features']
response12 = get_responses('hex350_grid_obesity_1920')['hex350_grid_obesity_1920']['features']

#maps of wellbeing outcomes

response13 = get_responses('hex350_grid_depression_1920')['hex350_grid_depression_1920']['features']
response14 = get_responses('hex350_grid_mental_1920')['hex350_grid_mental_1920']['features']
response15 = get_responses('hex350_grid_demantia_1920')['hex350_grid_demantia_1920']['features']

#maps of socioeconomic groups

response16 = get_responses('hex350_grid_popden_2020')['hex350_grid_popden_2020']['features']
response17 = get_responses('hex350_grid_gender2020')['hex350_grid_gender2020']['features']
response18 = get_responses('hex350_grid_age2020')['hex350_grid_age2020']['features']

#maps of health behaviours 

response19 = get_responses('hex350_grid_active2020')['hex350_grid_active2020']['features']
response20 = get_responses('hex350_grid_crime_22')['hex350_grid_crime_22']['features']

In [2]:
import geopandas as gpd 
import matplotlib.pyplot as plt 
import numpy as np

In [3]:
from branca.element import MacroElement
from jinja2 import Template

class BindColormap(MacroElement):
    def __init__(self, layer, colormap):
        super(BindColormap, self).__init__()
        self.layer = layer
        self.colormap = colormap
        self._template = Template(u"""
        {% macro script(this, kwargs) %}
            {{this.colormap.get_name()}}.svg[0][0].style.display = 'block';
            {{this._parent.get_name()}}.on('layeradd', function (eventLayer) {
                if (eventLayer.layer == {{this.layer.get_name()}}) {
                    {{this.colormap.get_name()}}.svg[0][0].style.display = 'block';
                }});
            {{this._parent.get_name()}}.on('layerremove', function (eventLayer) {
                if (eventLayer.layer == {{this.layer.get_name()}}) {
                    {{this.colormap.get_name()}}.svg[0][0].style.display = 'none';
                }});
        {% endmacro %}
        """)    

In [4]:
import folium
from folium import plugins
from folium import Map, FeatureGroup, Marker, LayerControl
import branca.colormap as cm
import branca

In [5]:
# Adding the layers
def hexmap (layer,color,feature_group):
    for feature in layer:
        properties = feature['properties']
        if '_mean' in properties:
            geoj = folium.GeoJson(feature, style_function=color)
            folium.features.GeoJsonPopup(fields=['_mean'], aliases=['Mean Value'], labels=True, localize=True).add_to(geoj)
            feature_group.add_child(geoj)
            
def hexmap2(layer,color,feature_group):
    for feature in layer:
        properties = feature['properties']
        if 'wprev_mean' in properties:
            geoj = folium.GeoJson(feature, style_function=color)
            folium.features.GeoJsonPopup(fields=['wprev_mean'], aliases=['Mean Value'], labels=True, localize=True).add_to(geoj)
            feature_group.add_child(geoj)
            
def hexmap3(layer, color, feature_group):
    for feature in layer:
        properties = feature['properties']
        if 'PROB_4BAND' in properties:
            geoj = folium.GeoJson(feature, style_function = color)
            folium.features.GeoJsonPopup(fields = ['PROB_4BAND'], aliases = ['Mean Value'], labels = True, localize = True).add_to(geoj)
            feature_group.add_child(geoj)
    
            
#ndvi

color_ndvi = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['_mean'] < 0.1 else 
    '#A2FF9E' if 0.1 <= x['properties']['_mean'] < 0.2 else 
    '#60C957' if 0.2 <= x['properties']['_mean'] < 0.4 else 
    '#008000'if 0.4 <= x['properties']['_mean'] < 0.6 else
    '#004D00','color':'white', 'weight':1, 'fillOpacity': '0.7'}
fg1 = folium.FeatureGroup(name='Normalized difference vegetation index (NDVI)',overlay=True, control=False)
hexmap(response,color_ndvi,fg1)       
scale_fg1 = branca.colormap.StepColormap(['#A2FF9E','#60C957','#008000','#004D00'],index=[0.10,0.24,0.42,0.60,0.78], vmin=0.10, vmax=0.78, caption='Normalized Difference Vegetation Index (NDVI > 0.1)')       

#pm25

color_pm25 = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['_mean'] < 9.95 else 
    '#E6F5FF' if 9.92<= x['properties']['_mean'] < 10.95 else 
    '#B2D4FF' if 10.95 <= x['properties']['_mean'] < 11.97 else 
    '#8A7FFF' if 11.97 <= x['properties']['_mean'] < 13 else
    '#9370DB' if 13.00 <= x['properties']['_mean'] < 14.02 else
    '#660099', 'color':'white', 'weight':1, 'fillOpacity': '0.7'}
fg2 = folium.FeatureGroup(name='Annual mean PM2.5 based on output areas average concentrations',overlay=True, control=False)
hexmap(response2, color_pm25, fg2)
scale_fg2 = branca.colormap.StepColormap(['#E6F5FF','#B2D4FF','#8A7FFF','#9370DB','#660099'], index=[9.95,10.95,11.97,13,14.02,15.05], vmin=9.95, vmax=15.05, caption='PM2.5 (μg/m³)(> 9.92)')

#pm10

color_pm10 = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['_mean'] < 2.71 else 
    '#E6F5FF' if 2.71 <= x['properties']['_mean'] < 9.21 else
    '#B2D4FF' if 9.21 <= x['properties']['_mean'] < 15.70 else
    '#8A7FFF' if 15.70 <= x['properties']['_mean'] < 22.20 else
    '#9370DB' if 22.20 <= x['properties']['_mean'] < 28.70 else
    '#660099', 'color':'white', 'weight':1, 'fillOpacity': '0.7'}
fg3 = folium.FeatureGroup(name='Annual mean PM10 based on output areas average concentrations',overlay=True, control=False)
hexmap(response3, color_pm10, fg3)
scale_fg3 = branca.colormap.StepColormap(['#E6F5FF','#B2D4FF','#8A7FFF','#9370DB','#660099'], index=[2.71,9.21,15.70,22.20,28.70,35.19], vmin=2.71, vmax=35.19, caption='PM10 (μg/m³)(> 2.71)')

#no2

color_no2 = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['_mean'] < 20 else 
    '#E6F5FF' if 20<= x['properties']['_mean'] < 25 else 
    '#B2D4FF' if 25 <= x['properties']['_mean'] < 30 else 
    '#8A7FFF' if 30 <= x['properties']['_mean'] < 35 else
    '#9370DB' if 35 <= x['properties']['_mean'] < 40 else
    '#660099', 'color':'white', 'weight':1, 'fillOpacity': '0.7'}

fg4 = folium.FeatureGroup(name='Annual mean NO2 concentrations',overlay=True, control=False)
hexmap(response4, color_no2, fg4)
scale_fg4 = branca.colormap.StepColormap(['#E6F5FF','#B2D4FF','#8A7FFF','#9370DB','#660099'], index=[20,25,30,35,40,45], vmin=20, vmax=45, caption='NO2 (μg/m³)')

#noise levels

color_noise = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['_mean'] is not None and x['properties']['_mean'] < 55 else
    '#FFFFCC' if x['properties']['_mean'] is not None and 55 <= x['properties']['_mean'] < 60 else             
    '#FFCC99' if x['properties']['_mean'] is not None and 60 <= x['properties']['_mean'] < 65 else
    '#FF9933' if x['properties']['_mean'] is not None and 65 <= x['properties']['_mean'] < 70 else
    '#FF6633' if x['properties']['_mean'] is not None else None,
    'color': 'white',
    'weight': 1,
    'fillOpacity': '0.7'}
fg5 = folium.FeatureGroup(name='Annual average noise levels of rail noise',overlay=True, control=False)
hexmap(response5, color_noise, fg5)
scale_fg5 = branca.colormap.StepColormap(['#FFFFCC','#FFCC99','#FF9933','#FF6633'], index=[55,60,65,70,75], vmin=55, vmax=75, caption='Noise levels (dB)(> 55)')

#flooding

#color_flood = lambda x: {
    #'fillColor': '#FFFFFF' if x['properties']['PROB_4BAND'] is None else
    #'#FFFFCC' if x['properties']['PROB_4BAND'] is not None and x['properties']['PROB_4BAND'] == 'Very Low' else             
    #'#FFCC99' if x['properties']['PROB_4BAND'] is not None and x['properties']['PROB_4BAND'] == 'Low' else
    #'#FF9933' if x['properties']['PROB_4BAND'] is not None and x['properties']['PROB_4BAND'] == 'Medium' else
    #'#FF6633' if x['properties']['PROB_4BAND'] is not None and x['properties']['PROB_4BAND'] == 'High' else
    #'#FFFFFF',
    #'color': 'white',
    #'weight': 1,
    #'fillOpacity': '0.7'}
fg6 = folium.FeatureGroup(name='Risk of flooding from rivers and seas',overlay=True, control=False)
#hexmap3(response6,color_flood,fg6)
scale_fg6 = branca.colormap.StepColormap(['#FFFFCC','#FFCC99','#FF9933','#FF6633'], index=[0,25,50,75,100], vmin=0, vmax=100, caption='Flood likelihoods')

#summer heat

color_heat = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['_mean'] is not None and x['properties']['_mean'] < 24.16 else
    '#0066CC' if x['properties']['_mean'] is not None and 24.16 <= x['properties']['_mean'] < 26.68 else
    '#ADD8E6' if x['properties']['_mean'] is not None and 26.68 <= x['properties']['_mean'] < 29.20 else 
    '#FFFF99' if x['properties']['_mean'] is not None and 29.20 <= x['properties']['_mean'] < 31.72 else
    '#FFA500' if x['properties']['_mean'] is not None and 31.72 <= x['properties']['_mean'] < 34.24 else
    '#FF0000', 'color':'white', 'weight':1, 'fillOpacity': '0.7'}

fg7 = folium.FeatureGroup(name='Major summer heat spots',overlay=True, control=False)
hexmap(response7, color_heat, fg7)
scale_fg7 = branca.colormap.StepColormap(['#0066CC','#ADD8E6','#FFFF99','#FFA500','#FF0000'], index=[24.16,26.68,29.20,31.72,34.24,36.76], vmin=24.16, vmax=36.76, caption='Mean daytime surface temperature (in °C)(> 24.16)')

#cycling

color_cycling = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['_mean'] is not None and x['properties']['_mean'] < 294.6 else
    '#FFF5F5' if x['properties']['_mean'] is not None and 294.6  <= x['properties']['_mean'] < 683.9 else
    '#ADD8E6' if x['properties']['_mean'] is not None and 683.9  <= x['properties']['_mean'] < 1073.3 else 
    '#66A3FF' if x['properties']['_mean'] is not None and 1073.3 <= x['properties']['_mean'] < 1462.6 else
    '#00BFA5' if x['properties']['_mean'] is not None and 1462.6 <= x['properties']['_mean'] < 1852 else
    '#008080' if x['properties']['_mean'] is not None and 1852   <= x['properties']['_mean'] < 2241.4 else
    '#004D00', 'color':'white', 'weight':1, 'fillOpacity': '0.7'}

fg8 = folium.FeatureGroup(name='Cycling routes density map',overlay=True, control=False)
hexmap(response8, color_cycling, fg8)
scale_fg8 = branca.colormap.StepColormap(['#FFF5F5','#ADD8E6','#66A3FF','#00BFA5','#008080','#004D00'], index=[294.6,683.9,1073.3,1462.6,1852,2241.4, 2630.7], vmin=294.6, vmax= 2630.7, caption = 'Length of cycling routes per sq.km (meter)(> 294.6)')

#land use

color_landuse = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['_mean'] is not None and x['properties']['_mean'] < 1.82 else
    '#FFFFCC' if x['properties']['_mean'] is not None and 1.82 <= x['properties']['_mean'] < 1.88 else
    '#FFCC99' if x['properties']['_mean'] is not None and 1.88 <= x['properties']['_mean'] < 1.95 else 
    '#FFA500' if x['properties']['_mean'] is not None and 1.95 <= x['properties']['_mean'] < 2.01 else
    '#FF6347' if x['properties']['_mean'] is not None and 2.01 <= x['properties']['_mean'] < 2.08 else
    '#8B0000', 'color':'white', 'weight':1, 'fillOpacity': '0.7'}

fg9 = folium.FeatureGroup(name='Land use diversity',overlay=True, control=False)
hexmap(response9, color_landuse, fg9)
scale_fg9 = branca.colormap.StepColormap (['#FFFFCC','#FFCC99','#FFA500','#FF6347','#8B0000'], index=[1.82,1.88,1.95,2.01,2.08,2.14], vmin=1.82, vmax=2.14, caption ='Shannon Entropy Diversity Index (> 1.82)')

#public space 

color_public = lambda x: {
    'fillColor': '#FFFF99' if x['properties']['green_infrastructure_metric_table metric_table_cln_composite_score_prop'] is not None and 0 <= x['properties']['green_infrastructure_metric_table metric_table_cln_composite_score_prop'] < 0.20 else
    '#90EE90' if x['properties']['green_infrastructure_metric_table metric_table_cln_composite_score_prop'] is not None and 0.20 <= x['properties']['green_infrastructure_metric_table metric_table_cln_composite_score_prop'] < 0.40 else 
    '#7FFF00' if x['properties']['green_infrastructure_metric_table metric_table_cln_composite_score_prop'] is not None and 0.40 <= x['properties']['green_infrastructure_metric_table metric_table_cln_composite_score_prop'] < 0.60 else
    '#008000' if x['properties']['green_infrastructure_metric_table metric_table_cln_composite_score_prop'] is not None and 0.60 <= x['properties']['green_infrastructure_metric_table metric_table_cln_composite_score_prop'] < 0.80 else
    '#003300', 'color':'white', 'weight':1, 'fillOpacity': '0.7'}

fg10 = folium.FeatureGroup(name='Proportion of population with access to public open space',overlay=True, control=False)

for feature in response10:
        properties = feature['properties']
        if 'green_infrastructure_metric_table metric_table_cln_composite_score_prop' in properties:
            geoj = folium.GeoJson(feature, style_function=color_public)
            folium.features.GeoJsonPopup(fields=['green_infrastructure_metric_table metric_table_cln_composite_score_prop'], aliases=['Mean Value'], labels=True, localize=True).add_to(geoj)
            fg10.add_child(geoj)

scale_fg10 = branca.colormap.StepColormap(['#FFFF99','#90EE90','#7FFF00','#008000','#003300'], index=[0,0.2,0.4,0.6,0.8,1], vmin=0, vmax=1, caption='Proportion')

#cardio

color_cardio = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['wprev_mean'] is not None and x['properties']['wprev_mean'] < 0.1573 else
    '#FFFF99' if x['properties']['wprev_mean'] is not None and 0.1573 <= x['properties']['wprev_mean'] < 0.8264 else
    '#FF6347' if x['properties']['wprev_mean'] is not None and 0.8264 <= x['properties']['wprev_mean'] < 1.4956 else 
    '#AA33AA' if x['properties']['wprev_mean'] is not None and 1.4956 <= x['properties']['wprev_mean'] < 2.1648 else
    '#4B0082' if x['properties']['wprev_mean'] is not None and 2.1648 <= x['properties']['wprev_mean'] < 2.8339 else
    '#000000', 'color':'white', 'weight':1, 'fillOpacity': '0.7'} 

fg11 = folium.FeatureGroup(name='Prevalence rates of cardiovascular diseases',overlay=True, control=False)
hexmap2(response11,color_cardio,fg11)
scale_fg11 = branca.colormap.StepColormap(['#FFFF99','#FF6347','#AA33AA','#4B0082','#000000'], index=[0.1573,0.8264,1.4956,2.1648,2.8339,3.5031], vmin=0.1573, vmax=3.5031, caption='Prevalence rate')

#obesity

color_obesity = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['wprev_mean'] is not None and x['properties']['wprev_mean'] < 1.1 else
    '#FFFF99' if x['properties']['wprev_mean'] is not None and 1.1 <= x['properties']['wprev_mean'] < 4.3 else
    '#FF6347' if x['properties']['wprev_mean'] is not None and 4.3 <= x['properties']['wprev_mean'] < 7.5 else 
    '#AA33AA' if x['properties']['wprev_mean'] is not None and 7.5 <= x['properties']['wprev_mean'] < 10.7 else
    '#4B0082' if x['properties']['wprev_mean'] is not None and 10.7 <= x['properties']['wprev_mean'] < 14 else
    '#000000', 'color':'white', 'weight':1, 'fillOpacity': '0.7'} 

fg12 = folium.FeatureGroup(name='Prevalence rates of obesity',overlay=True, control=False)
hexmap2(response12,color_obesity,fg12)
scale_fg12 = branca.colormap.StepColormap(['#FFFF99','#FF6347','#AA33AA','#4B0082','#000000'], index=[1.1,4.3,7.5,10.7,14,17.2], vmin=1.1, vmax=17.2, caption='Prevalence rate')


#depression

color_depression = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['wprev_mean'] is not None and x['properties']['wprev_mean'] < 1.9 else
    '#CCFFCC' if x['properties']['wprev_mean'] is not None and 1.9 <= x['properties']['wprev_mean'] < 4.3 else
    '#20B2AA' if x['properties']['wprev_mean'] is not None and 4.3 <= x['properties']['wprev_mean'] < 6.7 else 
    '#007F92' if x['properties']['wprev_mean'] is not None and 6.7 <= x['properties']['wprev_mean'] < 9.2 else
    '#20002C' if x['properties']['wprev_mean'] is not None and 9.2 <= x['properties']['wprev_mean'] < 11.6 else
    '#000000', 'color':'white', 'weight':1, 'fillOpacity': '0.7'} 

fg13 = folium.FeatureGroup(name='Prevalence rates of depression',overlay=True, control=False)
hexmap2(response13, color_depression, fg13)
scale_fg13 = branca.colormap.StepColormap(['#CCFFCC','#20B2AA','#007F92','#20002C','#000000'], index = [1.9,4.3,6.7,9.2,11.6,14.1], vmin=1.9, vmax=14.1, caption='Prevalence')

#mental

color_mental = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['wprev_mean'] is not None and x['properties']['wprev_mean'] < 0.26 else
    '#CCFFCC' if x['properties']['wprev_mean'] is not None and 0.26 <= x['properties']['wprev_mean'] < 0.75 else
    '#20B2AA' if x['properties']['wprev_mean'] is not None and 0.75 <= x['properties']['wprev_mean'] < 1.23 else 
    '#007F92' if x['properties']['wprev_mean'] is not None and 1.23 <= x['properties']['wprev_mean'] < 1.71 else
    '#20002C' if x['properties']['wprev_mean'] is not None and 1.71 <= x['properties']['wprev_mean'] < 2.2 else
    '#000000', 'color':'white', 'weight':1, 'fillOpacity': '0.7'}

fg14 = folium.FeatureGroup(name='Prevalence rates of mental health issues',overlay=True, control=False)
hexmap2(response14,color_mental,fg14)
scale_fg14 = branca.colormap.StepColormap(['#CCFFCC','#20B2AA','#007F92','#20002C','#000000'], index = [0.26,0.75,1.23,1.71,2.2,2.68], vmin=0.26, vmax=2.68, caption='Prevalence')

#dementia

color_dementia = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['wprev_mean'] is not None and x['properties']['wprev_mean'] < 0.06 else
    '#CCFFCC' if x['properties']['wprev_mean'] is not None and 0.06 <= x['properties']['wprev_mean'] < 0.53 else
    '#20B2AA' if x['properties']['wprev_mean'] is not None and 0.53 <= x['properties']['wprev_mean'] < 1 else 
    '#007F92' if x['properties']['wprev_mean'] is not None and 1 <= x['properties']['wprev_mean'] < 1.48 else
    '#20002C' if x['properties']['wprev_mean'] is not None and 1.48 <= x['properties']['wprev_mean'] < 1.95 else
    '#000000', 'color':'white', 'weight':1, 'fillOpacity': '0.7'}

fg15 = folium.FeatureGroup(name='Prevalence rates of dementia',overlay=True, control=False)
hexmap2(response15, color_dementia, fg15)
scale_fg15 = branca.colormap.StepColormap(['#CCFFCC','#20B2AA','#007F92','#20002C','#000000'], index = [0.06,0.53,1,1.48,1.95,2.42], vmin=0.06, vmax=2.42, caption='Prevalence')

#population

color_pop = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['_mean'] is not None and x['properties']['_mean'] < 439 else
    '#FFFF99' if x['properties']['_mean'] is not None and 439 <= x['properties']['_mean'] < 14016 else
    '#FF6347' if x['properties']['_mean'] is not None and 14016 <= x['properties']['_mean'] < 27592 else 
    '#AA33AA' if x['properties']['_mean'] is not None and 27592 <= x['properties']['_mean'] < 41169 else
    '#4B0082' if x['properties']['_mean'] is not None and 41169 <= x['properties']['_mean'] < 54745 else
    '#000000', 'color':'white', 'weight':1, 'fillOpacity': '0.7'} 

fg16 = folium.FeatureGroup(name='Population density map',overlay=True, control=False)
hexmap(response16, color_pop, fg16)
scale_fg16 = branca.colormap.StepColormap(['#FFFF99','#FF6347','#AA33AA','#4B0082','#000000'], index=[440,14000,28000,42000,55000], vmin=440, vmax=55000, caption='Population density (per sq km)')

#male ratio

color_male =  lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['_mean'] is not None and x['properties']['_mean'] < 73.68 else
    '#FF0000' if x['properties']['_mean'] is not None and 73.89 <= x['properties']['_mean'] < 82.45 else
    '#FF6666' if x['properties']['_mean'] is not None and 82.45 <= x['properties']['_mean'] < 91.23 else 
    '#FFCCCC' if x['properties']['_mean'] is not None and 91.23 <= x['properties']['_mean'] < 100 else
    '#CCDDFF' if x['properties']['_mean'] is not None and 100 <= x['properties']['_mean'] < 108.77 else
    '#87CEEB' if x['properties']['_mean'] is not None and 108.77 <= x['properties']['_mean'] < 117.55 else
    '#0000CD', 'color':'white', 'weight':1, 'fillOpacity': '0.7'}

fg17 = folium.FeatureGroup(name='Gender ratio map',overlay=True, control=False)
hexmap(response17,color_male,fg17)
scale_fg17 = branca.colormap.StepColormap(['#FF0000','#FF6666','#FFCCCC','#CCDDFF','#87CEEB','#0000CD'], index=[73.68, 82.45,91.23,100,108.77,117.55,236.81], vmin=73.86, vmax=236.81, caption='Number of males per 100 females')

#elder 

color_elder = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['_mean'] is not None and x['properties']['_mean'] < 0.01 else
    '#FFFF99' if x['properties']['_mean'] is not None and 0.01 <= x['properties']['_mean'] < 0.09 else
    '#D2B48C' if x['properties']['_mean'] is not None and 0.09 <= x['properties']['_mean'] < 0.17 else 
    '#808080' if x['properties']['_mean'] is not None and 0.17 <= x['properties']['_mean'] < 0.25 else
    '#36454F' if x['properties']['_mean'] is not None and 0.25 <= x['properties']['_mean'] < 0.33 else
    '#000022', 'color':'white', 'weight':1, 'fillOpacity': '0.7'}

fg18 = folium.FeatureGroup(name='Map of the ratio of elder people',overlay=True, control=False)
hexmap(response18, color_elder, fg18)
scale_fg18 = branca.colormap.StepColormap(['#FFFF99','#D2B48C','#808080','#36454F','#000022'], index=[0.01,0.09,0.17,0.25,0.33,0.40], vmin= 0.01, vmax=0.40, caption='Ratio of people over 65 years old')

#active people

color_active = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['_mean'] is not None and x['properties']['_mean'] < 62.60 else
    '#F5F5F5' if x['properties']['_mean'] is not None and 62.60 <= x['properties']['_mean'] < 67.12 else
    '#BFEFFF' if x['properties']['_mean'] is not None and 67.12 <= x['properties']['_mean'] < 71.64 else 
    '#87CEEB' if x['properties']['_mean'] is not None and 71.64 <= x['properties']['_mean'] < 76.16 else
    '#6495ED' if x['properties']['_mean'] is not None and 76.16 <= x['properties']['_mean'] < 80.68 else
    '#0000CD', 'color':'white', 'weight':1, 'fillOpacity': '0.7'}

fg19 = folium.FeatureGroup(name='Map of ratio of active people', overlay=True, control=False)
hexmap(response19, color_active,fg19)
scale_fg19 = branca.colormap.StepColormap(['#F5F5F5','#BFEFFF','#87CEEB','#6495ED','#0000CD'], index=[62.60,67.12,71.64,76.16,80.68], vmin=62.60, vmax=80.68, caption='Ratio of active people')

#crimes

color_crime = lambda x: {
    'fillColor': '#FFFFFF' if x['properties']['_mean'] is not None and x['properties']['_mean'] < 1 else
    '#FFFFE0' if x['properties']['_mean'] is not None and 1 <= x['properties']['_mean'] < 11 else
    '#FFD39B' if x['properties']['_mean'] is not None and 11 <= x['properties']['_mean'] < 21 else 
    '#FF7F50' if x['properties']['_mean'] is not None and 21 <= x['properties']['_mean'] < 31 else
    '#FF6347' if x['properties']['_mean'] is not None and 31 <= x['properties']['_mean'] < 41 else
    '#800080' if x['properties']['_mean'] is not None and 41 <= x['properties']['_mean'] < 51 else
    '#6A0DAD' if x['properties']['_mean'] is not None and 51 <= x['properties']['_mean'] < 61 else
    '#371F8F' if x['properties']['_mean'] is not None and 61 <= x['properties']['_mean'] < 71 else
    '#6495ED' if x['properties']['_mean'] is not None and 71 <= x['properties']['_mean'] < 81 else
    '#000033' if x['properties']['_mean'] is not None and 81 <= x['properties']['_mean'] < 91 else
    '#000000', 'color':'white', 'weight':1, 'fillOpacity': '0.7'}

fg20 = folium.FeatureGroup(name='Map of number of recorded crimes',overlay=True, control=False)
hexmap(response20, color_crime, fg20)
scale_fg20 = branca.colormap.StepColormap(['#FFFFE0','#FFD39B','#FF7F50','#FF6347','#800080','#6A0DAD','#371F8F','#6495ED','#000033','#000000'], index= [1,11,21,31,41,51,61,71,81,91,101], vmin=1, vmax=101, caption='Crime')


In [6]:
# Add each layer to the map and save it as a separate HTML file

for i in range(1, 21):
    # Create a new map object for each iteration
    m = folium.Map(location=[51.502264, -0.062134], control_scale=True, zoom_start=12, tiles=None)
    
    base_map1 = folium.FeatureGroup(name='OpenStreetMap', overlay=False)
    folium.TileLayer(tiles='OpenStreetMap').add_to(base_map1)
    # Add satellite imagery basemap
    base_map2 = folium.FeatureGroup(name="Esri World Imagery", overlay=False)
    folium.TileLayer(
        tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr='Esri',
        name='Esri Satellite'
    ).add_to(base_map2)

    base_map1.add_to(m)
    base_map2.add_to(m)
    
    fg_name = "fg" + str(i)
    fg = globals()[fg_name]
    m.add_child(fg)
    
    scale_name = "scale_fg" + str(i)
    scale = globals()[scale_name]
    m.add_child(scale)
    m.add_child(BindColormap(fg, scale))

    # Add LayerControl to toggle between basemaps
    folium.LayerControl(position='bottomleft').add_to(m)
    
    map_fname = f'Output_London_Layer{i}.html'
    m.save(map_fname)